In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from xgboost import XGBRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#pip install xgboost

In [ ]:
# Read csv files. And keep a copy of the original dataframes. tran_df and test_df will be used throughout this notebook for preprocessing and scaling.we need original test dataframe in the end
or_train_df = pd.read_csv('train.csv')
or_test_df = pd.read_csv('test.csv')

train_df = or_train_df
test_df = or_test_df

In [ ]:
# Dropping 'Id' column as it has no use in training
train_df = train_df.drop(['Id'], axis=1)
test_df = test_df.drop(['Id'], axis=1)

In [ ]:
print(f"Shape of train set: {train_df.shape}")
print(f"Shape of test set: {test_df.shape}")

## Replace NaNs
**While replacing we have to make sure to replace it with '0' (character) in case of a string column and 0 (number) if otherwise**

In [ ]:
str_columns = []
num_columns = []

def convert_to_str(df):
    for col in df.columns:
        if (df[col].dtype == np.int64 or df[col].dtype == np.float64):
            df[col] = df[col].fillna(method='ffill')
            if col not in num_columns:
                num_columns.append(col)
        else:
            df[col] = df[col].fillna(method='ffill')
            if col not in str_columns:
                str_columns.append(col)
    return df

train_df = convert_to_str(train_df)
test_df = convert_to_str(test_df)

In [ ]:
train_df['type'] = 'train'
test_df['type'] = 'test'

# Add a dummy SalePrice column to test dataframe to make number of columns equal
test_df['SalePrice'] = train_df['SalePrice'].iloc[:201]

df = train_df._append(test_df, ignore_index=True)

In [ ]:
for col in str_columns:
    one_hot = pd.get_dummies(df[col])

    replace_cols = {}
    for one_col in one_hot.columns:
        replace_cols[one_col] = f"{col}_{one_col}"
    one_hot = one_hot.rename(columns=replace_cols)

    df = df.drop(col, axis = 1)
    df = df.join(one_hot)

In [ ]:
train_df = df[df['type'] == 'train']
test_df = df[df['type'] == 'test']

train_df = train_df.drop(['type'], axis=1)
test_df = test_df.drop(['type'], axis=1)

test_df = test_df.reset_index(drop=True)

In [ ]:
print(f"Shape of train set: {train_df.shape}")
print(f"Shape of test set: {test_df.shape}")

## Standardization of values

In [ ]:
scaler = StandardScaler()
scaler.fit(train_df[num_columns])

In [ ]:
train_df[num_columns] = scaler.transform(train_df[num_columns])
test_df[num_columns] = scaler.transform(test_df[num_columns])

In [ ]:
print(f"Shape of train set: {train_df.shape}")
print(f"Shape of test set: {test_df.shape}")

In [ ]:
test_df = test_df.drop(['SalePrice'], axis=1)

In [ ]:
train_labels = train_df['SalePrice']
train_data = train_df.drop(['SalePrice'], axis=1)

## XGBoost regressor model

In [ ]:
model = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

In [ ]:
model.fit(train_data, train_labels)

In [ ]:
predictions = model.predict(test_df)

In [ ]:
test_df['SalePrice'] = predictions
test_df[num_columns] = scaler.inverse_transform(test_df[num_columns])
test_df = pd.DataFrame(test_df, columns=train_df.columns)

In [ ]:
# Create results dataframe
results = pd.DataFrame()
results['Id'] = or_test_df['Id']
results['SalePrice'] = test_df['SalePrice']

In [ ]:
results.head()

In [ ]:
results.to_csv('submissions.csv', index=False)